In [ ]:
# Simple CA simulator in Python
#
# *** Hosts & Pathogens ***
#
# Copyright 2008-2012 Hiroki Sayama
# sayama@binghamton.edu

# Modified to run with Python 3

import matplotlib
matplotlib.use('TkAgg')

import pylab as PL
import random as RD
import scipy as SP

RD.seed()

width = 50
height = 50
initProb = 0.2
infectionRate = 0.3
regrowthRate = 0.1

def init():
    global time, config, nextConfig

    time = 0

    config = SP.zeros([height, width])
    for x in range(width):
        for y in range(height):
            if RD.random() < initProb:
                state = 2
            else:
                state = 1
            config[y, x] = state

    nextConfig = SP.zeros([height, width])

def draw():
    PL.cla()
    PL.pcolor(config, vmin = 0, vmax = 2, cmap = PL.cm.jet)
    PL.axis('image')
    PL.title('t = ' + str(time))

def step():
    global time, config, nextConfig

    time += 1

    step_state_count = [0, 0, 0]

    for x in range(width):
        for y in range(height):
            state = config[y, x]
            if state == 0:
                for dx in range(-1, 2):
                    for dy in range(-1, 2):
                        if config[(y+dy)%height, (x+dx)%width] == 1:
                            if RD.random() < regrowthRate:
                                state = 1
            elif state == 1:
                for dx in range(-1, 2):
                    for dy in range(-1, 2):
                        if config[(y+dy)%height, (x+dx)%width] == 2:
                            if RD.random() < infectionRate:
                                state = 2
            else:
                state = 0

            step_state_count[int(state)] += 1
            nextConfig[y, x] = state

    step_state_count = [num/width/height for num in step_state_count]
    state_counter.append(step_state_count)
    config, nextConfig = nextConfig, config

import pycxsimulator
state_counter = []

pycxsimulator.GUI().start(func=[init,draw,step])

x = list(range(len(state_counter)))
s_0 = [row[0] for row in state_counter]
s_1 = [row[1] for row in state_counter]
s_2 = [row[2] for row in state_counter]

fig, ax = matplotlib.pyplot.subplots()
ax.plot(x, s_0, 'b-', label="Empty")
ax.plot(x, s_1, 'g-', label="Healthy")
ax.plot(x, s_2, 'r-', label="Infected")
ax.set_xlabel('# of steps')
ax.set_ylabel('% of population')
ax.set_title("initProb={:.2f} infectionRate={:.2f} regrowthRate={:.2f}".format(initProb, infectionRate, regrowthRate))
ax.legend()
fig.savefig("initProb={:.2f}_infectionRate={:.2f}_regrowthRate={:.2f}.png".format(initProb, infectionRate, regrowthRate))
